In [41]:
pip install mysql-connector-python faker

In [59]:
import mysql.connector
import random
from faker import Faker
from datetime import datetime,timedelta

In [61]:
fake = Faker()

In [63]:
# Creating a connection to the database
DB_CONFIG = {
    "host": "localhost",
    "user":"root",
    "password":"root",
    "database":"crop_management"
}

In [65]:
# Connect to MySQL
conn = mysql.connector.connect(**DB_CONFIG)
cursor = conn.cursor()

In [67]:
# Create crops table if not exists
cursor.execute("""
CREATE TABLE IF NOT EXISTS crops (
    id INT AUTO_INCREMENT PRIMARY KEY,
    crop_name VARCHAR(255) NOT NULL,
    planting_date DATE,
    harvest_date DATE,
    growth_stage VARCHAR(255),
    pest_control_measures TEXT,
    yield_prediction INT
)
""")
conn.commit()

In [69]:
# List of sample crop names
crop_names = ["Wheat", "Rice", "Corn", "Soybean", "Barley", "Sugarcane", "Cotton", "Potato", "Tomato", "Lettuce"]

# List of possible growth stages
growth_stages = ["Seedling", "Vegetative", "Flowering", "Fruiting", "Maturity"]

# List of sample pest control measures
pest_control_measures_list = [
    "Use of organic pesticides",
    "Crop rotation",
    "Neem oil application",
    "Biological pest control",
    "Chemical pesticides",
    "Regular field monitoring",
]

In [71]:
# Function to generate random data for crops
def generate_data():
    crop_name = random.choice(crop_names)
    planting_date = fake.date_between(start_date="-2y", end_date="today")  # Planting in last 2 years
    harvest_date = planting_date + timedelta(days=random.randint(60, 180))  # Harvest after 2-6 months
    growth_stage = random.choice(growth_stages)
    pest_control = random.choice(pest_control_measures_list)
    yield_prediction = random.randint(500, 5000)  # Yield in kg
    
    return (crop_name, planting_date, harvest_date, growth_stage, pest_control, yield_prediction)

In [73]:
# Batch insert records in chunks
batch_size = 100000  # Insert 10,000 at a time for efficiency
total_records = 20000000

for i in range(0, total_records, batch_size):
    data_batch = [generate_data() for _ in range(batch_size)]
    
    # Execute batch insert
    cursor.executemany("""
        INSERT INTO crops (crop_name, planting_date, harvest_date, growth_stage, pest_control_measures, yield_prediction)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, data_batch)
    conn.commit()
    
    print(f"{i + batch_size} records inserted...")

print("✅ Data insertion completed successfully!")

# Close the database connection
cursor.close()
conn.close()

100000 records inserted...
200000 records inserted...
300000 records inserted...
400000 records inserted...
500000 records inserted...
600000 records inserted...
700000 records inserted...
800000 records inserted...
900000 records inserted...
1000000 records inserted...
1100000 records inserted...
1200000 records inserted...
1300000 records inserted...
1400000 records inserted...
1500000 records inserted...
1600000 records inserted...
1700000 records inserted...
1800000 records inserted...
1900000 records inserted...
2000000 records inserted...
2100000 records inserted...
2200000 records inserted...
2300000 records inserted...
2400000 records inserted...
2500000 records inserted...
2600000 records inserted...
2700000 records inserted...
2800000 records inserted...
2900000 records inserted...
3000000 records inserted...
3100000 records inserted...
3200000 records inserted...
3300000 records inserted...
3400000 records inserted...
3500000 records inserted...
3600000 records inserted...
3

In [75]:
import pandas as pd

In [77]:
def fetch_table_data(table_name):
    # The connect() constructor creates a connection to the MySQL server and returns a MySQLConnection object.
    cnx = mysql.connector.connect(
        host='localhost',
        database='crop_management',
        user='root',
        password='root'
    )

    cursor = cnx.cursor()
    cursor.execute('select * from ' + table_name)

    header = [row[0] for row in cursor.description]

    rows = cursor.fetchall()

    # Closing connection
    cnx.close()

    return header, rows
def export(table_name):
    header, rows = fetch_table_data(table_name)

    # Create csv file
    f = open(table_name + '.csv', 'w')

    # Write header
    f.write(','.join(header) + '\n')

    for row in rows:
        f.write(','.join(str(r) for r in row) + '\n')

    f.close()
    print(str(len(rows)) + ' rows written successfully to ' + f.name)


# Tables to be exported
export('crops')


20000000 rows written successfully to crops.csv


In [82]:
import pandas as pd
df = pd.read_csv("crops.csv")

In [79]:
df

,id,crop_name,planting_date,harvest_date,growth_stage,pest_control_measures,yield_prediction
0,1,Sugarcane,2023-06-06,2023-11-04,Flowering,Crop rotation,3180
1,2,Barley,2023-10-09,2024-03-14,Fruiting,Crop rotation,3435
2,3,Cotton,2024-12-08,2025-05-29,Vegetative,Neem oil application,4289
3,4,Potato,2024-01-19,2024-05-15,Flowering,Crop rotation,3495
4,5,Sugarcane,2023-07-11,2023-10-30,Maturity,Neem oil application,4342
...,...,...,...,...,...,...,...
19999995,19999996,Soybean,2024-05-26,2024-07-31,Fruiting,Chemical pesticides,3742
19999996,19999997,Cotton,2023-09-22,2023-12-12,Vegetative,Use of organic pesticides,3618
19999997,19999998,Wheat,2024-12-25,2025-06-12,Flowering,Chemical pesticides,2559
19999998,19999999,Lettuce,2024-03-15,2024-08-02,Fruiting,Crop rotation,3668


In [84]:
df.size

140000000

In [89]:
import os

file_path = "crops.csv"  # Replace with your actual file path
file_size = os.path.getsize(file_path)  # Size in bytes

print(f"File size: {file_size / (1024 * 1024):.2f} MB")  # Convert to MB


File size: 1422.88 MB
